In [1]:
import pandas as pd
import json
import json
from tqdm import tqdm
import pandas as pd
import sys
sys.path.append("../src")
import prompt_utils
import llm_utils
import os
import random
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# vicuna 
# with rules classification only (0.76)
vicuna_base_path = "../data/vicuna_4bit/"
vicuna_with_rules_classification_only_name = "generic_prompt_with_rules_only_classification"
vicuna_with_rules_classification_only_func = prompt_utils.get_vicuna_prompt_with_rules_only_classification

# OA LLAMA
# Classification Only V03 (0.81)
# 1 pos 1 neg (0.79)
oa_base_path = "../data/openassistant_llama_30b_4bit/"
oa_classification_only_v03_name = "generic_prompt_without_context_only_classification_v03"
oa_classification_only_v03_func = prompt_utils.get_openassistant_llama_30b_4bit_without_context_only_classification_v03
oa_1pos_1neg_path_name = "generic_prompt_few_shot_prompt_only_classification_1_pos_1_neg_example"
oa_1pos_1neg_path_func = prompt_utils.get_openassistant_llama_30b_4bit_few_shot_prompt_only_classification_1_pos_1_neg_example

# Text Davinci
# Elaboration First V02 (0.94)
davinci_base_path = "../data/openai_text_davinci_003/"
davinci_elaboration_first_v02_name = "generic_prompt_without_context_elaboration_first_v02"
davinci_elaboration_first_v02_func = prompt_utils.get_openai_prompt_without_context_elaboration_first_v02

In [2]:
classes = ["War/Terror", "Conspiracy Theory", "Education", "Election Campaign", "Environment", 
              "Government/Public", "Health", "Immigration/Integration", 
              "Justice/Crime", "Labor/Employment", 
              "Macroeconomics/Economic Regulation", "Media/Journalism", "Religion", "Science/Technology"]

def load_model(url, extraction_func, metrics_calculation_func):
    df = pd.read_csv(url)
    predictions_per_class, confusion_matrices, binary_classification_reports, multilabel_classification_reports = metrics_calculation_func(df, classes, extraction_func)
    data = {"confusion_matrices": confusion_matrices, "binary_classification_reports": binary_classification_reports, "multilabel_classification_reports": multilabel_classification_reports}
    return data, predictions_per_class

for i in range(0,5):
    binary_war_path = f"{vicuna_base_path}{vicuna_with_rules_classification_only_name}/generic_test_{i}.csv"
    print(binary_war_path)
    data, with_rules_classification_only_predictions_per_class = load_model(binary_war_path, llm_utils.get_extraction_function("extract_nth_character", 2), llm_utils.calculate_binary_metrics)
    print(data["binary_classification_reports"])

../data/vicuna_4bit/generic_prompt_with_rules_only_classification/generic_test_0.csv
                                 label  f1_score_macro  precision_macro  recall_macro support_macro  f1_score_class_0 support_class_0  f1_score_class_1 support_class_1  precision_class_0  recall_class_0  precision_class_1  recall_class_1
0                           War/Terror        0.762427         0.804063      0.769231           130          0.802632              65          0.722222              65           0.701149        0.938462           0.906977        0.600000
1                    Conspiracy Theory        0.519046         0.686101      0.584615           130          0.696629              65          0.341463              65           0.548673        0.953846           0.823529        0.215385
2                            Education        0.758004         0.830573      0.769231           130          0.810127              65          0.705882              65           0.688172        0.98461

In [3]:
for i in range(0,2):
    path = f"{oa_base_path}{oa_classification_only_v03_name}/generic_test_{i}.csv"
    print(path)
    data, with_rules_classification_only_predictions_per_class = load_model(binary_war_path, llm_utils.get_extraction_function("extract_nth_character", 2), llm_utils.calculate_binary_metrics)
    print(data["binary_classification_reports"])

../data/openassistant_llama_30b_4bit/generic_prompt_without_context_only_classification_v03/generic_test_0.csv
                                 label  f1_score_macro  precision_macro  recall_macro support_macro  f1_score_class_0 support_class_0  f1_score_class_1 support_class_1  precision_class_0  recall_class_0  precision_class_1  recall_class_1
0                           War/Terror        0.762427         0.804063      0.769231           130          0.802632              65          0.722222              65           0.701149        0.938462           0.906977        0.600000
1                    Conspiracy Theory        0.519046         0.686101      0.584615           130          0.696629              65          0.341463              65           0.548673        0.953846           0.823529        0.215385
2                            Education        0.758004         0.830573      0.769231           130          0.810127              65          0.705882              65        

In [4]:
for i in range(0,5):
    binary_war_path = f"{vicuna_base_path}{vicuna_with_rules_classification_only_name}/generic_test_top_p_1_{i}.csv"
    print(binary_war_path)
    data, with_rules_classification_only_predictions_per_class = load_model(binary_war_path, llm_utils.get_extraction_function("extract_nth_character", 2), llm_utils.calculate_binary_metrics)
    print(data["binary_classification_reports"])

../data/vicuna_4bit/generic_prompt_with_rules_only_classification/generic_test_top_p_1_0.csv
                                 label  f1_score_macro  precision_macro  recall_macro support_macro  f1_score_class_0 support_class_0  f1_score_class_1 support_class_1  precision_class_0  recall_class_0  precision_class_1  recall_class_1
0                           War/Terror        0.788694         0.813770      0.792308           130          0.816327              65          0.761062              65           0.731707        0.923077           0.895833        0.661538
1                    Conspiracy Theory        0.528986         0.605178      0.569231           130          0.666667              65          0.391304              65           0.543689        0.861538           0.666667        0.276923
2                            Education        0.723750         0.803008      0.738462           130          0.787500              65          0.660000              65           0.663158       

In [5]:
prompt, followup, request_params = prompt_utils.get_openassistant_llama_30b_4bit_without_context_only_classification_v03("Hey how are you?", "War/Terror", prompt_utils.get_base_request_params())
#print(prompt)
        
#request_params['regenerate'] = False
#request_params['do_sample'] = False
request_params['max_new_tokens'] = 200
response = prompt_utils.get_response(request_params, prompt, "")
print(response)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/v1/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f82f9b33290>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import tstd

def calculate_fbeta_score(beta, precision, recall):
    return (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)

def generate_latex_table(df):
    latex_table = df.to_latex(index=False, float_format="%.2f")
    with open('latex_table.tex', 'w') as f:
        f.write(latex_table)

beta = 0.5
files = []
total_metrics_per_csv = []
total_metrics_per_csv_lower_occurance = []
low_f1_classes = ["Conspiracy Theory", "Education", "Environment", "Labor/Employment", "Science/Technology", "Religion"]
for i in range(0,5):
    binary_war_path = f"{vicuna_base_path}{vicuna_with_rules_classification_only_name}/generic_test_top_p_1_{i}.csv"
    data, with_rules_classification_only_predictions_per_class = load_model(binary_war_path, llm_utils.get_extraction_function("extract_nth_character", 2), llm_utils.calculate_binary_metrics)
    files.append(data["binary_classification_reports"])

    # Calculate the mean metrics for each label in this CSV
    mean_metrics = data["binary_classification_reports"].mean()
    mean_metrics_lower_occurance = data["binary_classification_reports"].set_index('label').loc[low_f1_classes].mean()
    total_metrics_per_csv_lower_occurance.append(mean_metrics_lower_occurance)
    total_metrics_per_csv.append(mean_metrics)

# Concatenate all dataframes
df = pd.concat(files)
df['f1_beta_score_class_1'] = df.apply(lambda row: calculate_fbeta_score(beta, row['precision_class_1'], row['recall_class_1']), axis=1)

# Group by label and calculate mean and standard deviation
grouped = df.groupby('label').agg(['mean', 'std'])

total_metrics = pd.concat(total_metrics_per_csv)
total_metrics = pd.DataFrame(total_metrics.values.reshape(-1, len(total_metrics)//5), columns=total_metrics.index[:len(total_metrics)//5])
total_metrics['f1_beta_score_class_1'] = total_metrics.apply(lambda row: calculate_fbeta_score(beta, row['precision_class_1'], row['recall_class_1']), axis=1)

total_metrics_lower_occurance = pd.concat(total_metrics_per_csv_lower_occurance)
total_metrics_lower_occurance = pd.DataFrame(total_metrics_lower_occurance.values.reshape(-1, len(total_metrics_lower_occurance)//5), columns=total_metrics_lower_occurance.index[:len(total_metrics_lower_occurance)//5])
total_metrics_lower_occurance['f1_beta_score_class_1'] = total_metrics_lower_occurance.apply(lambda row: calculate_fbeta_score(beta, row['precision_class_1'], row['recall_class_1']), axis=1)

total_metrics_grouped = total_metrics.agg(['mean', 'std'])
total_metrics_lower_occurance_grouped = total_metrics_lower_occurance.agg(['mean', 'std'])

for column in grouped.columns:
    if column[1] == 'mean':
        grouped.loc['Total', column] = total_metrics_grouped[column[0]][0]
        grouped.loc['Total Lower Occurrance', column] = total_metrics_lower_occurance_grouped[column[0]][0]
    else:
        grouped.loc['Total', column] = total_metrics_grouped[column[0]][1]
        grouped.loc['Total Lower Occurrance', column] = total_metrics_lower_occurance_grouped[column[0]][1]

# Select only the columns we're interested in
grouped = grouped[['precision_class_1', 'recall_class_1', 'f1_score_class_1', 'f1_beta_score_class_1']]

# Combine mean and std into one column
for column in grouped.columns:
    if column[1] == 'mean':
        grouped[column[0], 'result'] = grouped[column].apply(lambda x: f"{x:.2f}") + " ± " + grouped[(column[0], 'std')].apply(lambda x: f"{x:.2f}")


# Reset index to include labels as a column
grouped.reset_index(inplace=True)
grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]
grouped.rename(columns=lambda x: x.rstrip('_'), inplace=True)
grouped = grouped.filter(regex='^(?!.*_mean|.*_std).*$')
grouped.rename(columns=lambda x: x.replace('_result', ''), inplace=True)
print(grouped)
# Generate LaTeX table
generate_latex_table(grouped)

                                 label precision_class_1 recall_class_1 f1_score_class_1 f1_beta_score_class_1
0                    Conspiracy Theory       0.76 ± 0.06    0.27 ± 0.02      0.40 ± 0.03           0.56 ± 0.04
1                            Education       0.92 ± 0.02    0.59 ± 0.06      0.72 ± 0.04           0.83 ± 0.02
2                    Election Campaign       0.86 ± 0.03    0.59 ± 0.04      0.70 ± 0.03           0.79 ± 0.02
3                          Environment       0.78 ± 0.04    0.64 ± 0.02      0.71 ± 0.03           0.75 ± 0.03
4                    Government/Public       0.62 ± 0.03    0.48 ± 0.03      0.54 ± 0.03           0.58 ± 0.02
5                               Health       0.90 ± 0.07    0.45 ± 0.03      0.60 ± 0.04           0.75 ± 0.05
6              Immigration/Integration       0.89 ± 0.03    0.44 ± 0.05      0.59 ± 0.04           0.74 ± 0.03
7                        Justice/Crime       0.71 ± 0.04    0.48 ± 0.02      0.57 ± 0.02           0.65 ± 0.03
8

In [7]:
# Select the columns of interest and combine mean and std into one column
grouped = grouped[['precision_class_1', 'recall_class_1', 'f1_score_class_1', 'f1_beta_score_class_1']].apply(
    lambda x: x['mean'].apply(lambda y: f"{y:.2f}") + " ± " + x['std'].apply(lambda z: f"{z:.2f}")
)

# Reset index to include labels as a column and format column names
grouped.reset_index(inplace=True)
grouped.columns = ['_'.join(col).strip().replace('_mean', '').replace('_std', '') for col in grouped.columns.values]

print(grouped)


KeyError: 'mean'